# crawling page

## kode untuk mellihhat arsitektur struktur halaman web

In [20]:
!pip install builtwith

  Preparing metadata (setup.py) ... done
  Created wheel for builtwith: filename=builtwith-1.3.4-py3-none-any.whl size=36077 sha256=31934737992a5d70ecf64f784f53026390c6e7b463c9d4dc3181a8f5857652c9
  Stored in directory: /root/.cache/pip/wheels/7f/2d/b2/606e3df914d4aeeab99c4a4e3e9a61673d2293c2e346db00c8
Successfully built builtwith


In [21]:
import builtwith

# Analisis teknologi yang digunakan
res = builtwith.parse('https://pta.trunojoyo.ac.id')
print(res)

{'web-servers': ['Nginx'], 'javascript-frameworks': ['jQuery', 'jQuery UI']}


## Crawling Link data text

In [22]:
import requests
from bs4 import BeautifulSoup

def crawl_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        # Ambil semua judul h1, h2, h3
        headings = soup.find_all(['h1', 'h2', 'h3'])
        for heading in headings:
            print(f"{heading.name}: {heading.get_text()}")

        # Ambil semua link
        links = soup.find_all('a', href=True)
        for link in links:
            print(f"URL: {link['href']} | Teks: {link.get_text()}")

    except requests.exceptions.RequestException as e:
        print(f"Terjadi kesalahan saat mengakses {url}: {e}")

# Gunakan fungsi
crawl_website("https://trunojoyo.ac.id/")

h3:  Unggul
h3:  Tangguh
h3:  Mandiri
h2: Tentang UTM
h2: Excellence Base On Local Wisdom
h2: PIMPINAN UNIVERSITAS TRUNOJOYO MADURA
h3: Prof. Dr. Safi’, S.H., M.H.
h3: Prof. Dr. Achmad Amzeri, S.P., M.P.
h3: Ir. Ari Basuki, S.T., M.T.
h3: Surokim, S.Sos., S.H., M.Si.
h2: Sampaikan Saran Disini
h3: Laman Penunjang
h3: Alamat
h3: Email Address
h3: Telepon, WA
h3: Official Account
URL: https://www2.trunojoyo.ac.id/ | Teks: 


URL: tel:031-3011146 | Teks: 031-3011146
URL: mailto:humas@trunojoyo.ac.id | Teks: humas@trunojoyo.ac.id
URL: # | Teks:   
URL: #  | Teks:   
URL: #  | Teks:   
URL: #  | Teks:   
URL: # | Teks: Tentang
URL: https://www2.trunojoyo.ac.id/index.php/sejarahutm/ | Teks: Sejarah
URL: https://www2.trunojoyo.ac.id/index.php/profil-singkat/ | Teks: Profil Singkat
URL: https://www2.trunojoyo.ac.id/index.php/visimisi/ | Teks: Visi dan Misi
URL: https://www2.trunojoyo.ac.id/index.php/lhm/ | Teks: Logo, Hymne dan Mars
URL: https://www2.trunojoyo.ac.id/index.php/organ-utm/ | Teks

## kode untuk mengambil semua judul dan informasi yand diperlukan dari judul itu yang ada di pta.trunojoyo.ac.id dan khusus untuk fakultas teknik

In [1]:
!pip install requests
!pip install beautifulsoup4


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def ptaa(save_csv="pta_final_minimal.csv", pages_per_prodi=3, sleep_range=(0.3, 0.8)):
    daftar_prodi = {
        "Teknik Industri": "9",
        "Teknik Informatika": "10",
        "Manajemen Informatika": "11",
        "Teknik Multimedia Dan Jaringan": "19",
        "Mekatronika": "20",
        "Teknik Elektro": "23",
        "Sistem Informasi": "31",
        "Teknik Mesin": "32",
        "Teknik Mekatronika": "33"
    }

    data = {
        "penulis": [],
        "judul": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "abstrak": [],
        "prodi": [],
        "abstrak_inggris": []
    }

    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(['GET', 'POST'])
    )
    adapter = HTTPAdapter(max_retries=retries)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    session.headers.update({
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116 Safari/537.36"
    })

    def get_label_text(soup_parent, label_pattern):
        el = soup_parent.find(lambda tag: tag.name == "span" and re.search(label_pattern, tag.get_text(), re.I))
        if el:
            txt = el.get_text(separator=' ').split(':', 1)
            return txt[1].strip() if len(txt) > 1 else el.get_text().strip()
        return "Tidak tersedia"

    try:
        for prodi_nama, prodi_id in daftar_prodi.items():
            for page in range(1, pages_per_prodi + 1):
                list_url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi_id}/{page}"
                try:
                    r = session.get(list_url, timeout=(5, 12))
                    r.raise_for_status()
                except requests.exceptions.RequestException:
                    time.sleep(random.uniform(*sleep_range))
                    continue

                soup = BeautifulSoup(r.content, "lxml")
                jurnals = soup.select('li[data-cat="#luxury"]')
                if not jurnals:
                    time.sleep(random.uniform(*sleep_range))
                    continue

                for jurnal in jurnals:
                    a_tag = jurnal.select_one('a.gray.button')
                    if not a_tag or not a_tag.get('href'):
                        continue
                    link_detail = a_tag['href']
                    link_detail = requests.compat.urljoin(list_url, link_detail)

                    try:
                        response = session.get(link_detail, timeout=(5, 12))
                        response.raise_for_status()
                    except requests.exceptions.RequestException:
                        time.sleep(random.uniform(*sleep_range))
                        continue

                    soup1 = BeautifulSoup(response.content, "lxml")
                    isi = soup1.select_one('div#content_journal')
                    if not isi:
                        time.sleep(random.uniform(*sleep_range))
                        continue

                    judul_tag = isi.select_one('a.title')
                    judul = judul_tag.get_text(strip=True) if judul_tag else "Tidak tersedia"

                    penulis = get_label_text(isi, r'Penulis')
                    pembimbing_pertama = get_label_text(isi, r'Dosen Pembimbing I|Pembimbing I')
                    pembimbing_kedua = get_label_text(isi, r'Dosen Pembimbing II|Pembimbing II')

                    semua_abstrak = isi.select('p[align="justify"]')
                    abstrak = semua_abstrak[0].get_text(strip=True) if len(semua_abstrak) > 0 else "Tidak tersedia"
                    abstrak_inggris = semua_abstrak[1].get_text(strip=True) if len(semua_abstrak) > 1 else "Tidak tersedia"

                    data["penulis"].append(penulis)
                    data["judul"].append(judul)
                    data["pembimbing_pertama"].append(pembimbing_pertama)
                    data["pembimbing_kedua"].append(pembimbing_kedua)
                    data["abstrak"].append(abstrak)
                    data["prodi"].append(prodi_nama)
                    data["abstrak_inggris"].append(abstrak_inggris)

                    time.sleep(random.uniform(*sleep_range))

    except KeyboardInterrupt:
        df_partial = pd.DataFrame(data)
        df_partial.to_csv(save_csv.replace(".csv", "_partial.csv"), index=False)
        return df_partial

    df = pd.DataFrame(data)
    df.to_csv(save_csv, index=False)
    return df


In [17]:
ptaa()

,penulis,judul,pembimbing_pertama,pembimbing_kedua,abstrak,prodi,abstrak_inggris
0,Siliwangi Fitra Rachmawanto S.T.,OPTIMASI PEMILIHAN PORTOFOLIO SAHAM PERUSAHAAN...,"Heri Awalul Ilhamsah S.T., M.T.","Retno Indriartiningtias S.T., M.T.",Portofolio adalah sekumpulan saham yang dimili...,Teknik Industri,Portofolio is a collection of stock owned by i...
1,AHMAD MAS'UD,PERANCANGAN TATA LETAK FASILITAS LANTAI PRODUK...,"SABARUDIN AKHMAD, S.T., M.T.","SUGENG PURWOKO, S.T., M.T.",PT. ABC merupakan perusahaan yang bergerak dib...,Teknik Industri,PT. ABC is a company engaged in the manufactur...
2,Yulianto Fauzanta,PERUMUSAN STRATEGI BISNIS UD. BUDI JAYA BANGKA...,"Fitri Agustina, S.T., M.T","Retno Indriartiningtias, S.T., M.T",Bangkalan merupakan salah satu kabupaten yang ...,Teknik Industri,Bangkalan is one of the districts that have th...
3,M Mundir Muhlisin,USULAN PERBAIKAN UTILITAS RESOURCES PADA LANTA...,Mu'alim ST MT,Sugeng Purwoko ST MT,Simulasi adalah duplikasi atau abstraksi dari ...,Teknik Industri,Simulation is a duplication or abstraction of ...
4,Muhibbin,Peningkatan Kepuasan Masyarakat Terhadap Pelay...,Rahmad Hidayat,Retno Indriartiningtias,Kepuasan adalah tingkat perasaan seseorang ter...,Teknik Industri,Satisfaction is feeling level of someone to se...
...,...,...,...,...,...,...,...
99,MOH TAUFIK HIDAYAT,ALAT PENGGULUNG UNTUK MENGATUR KERAPATAN KERTA...,"FAIKUL UMAM., S.KOM., M.T","SRI WAHYUNI., S.KOM., M.T",ABSTRAK\n\nKertas merupakan kebutuhan yang dib...,Teknik Mekatronika,ABSTRACT\n\nPaper is a necessity needed by hum...
100,Alvian Ainun Fajrih,Optimasi Penguapan Air Laut Tua Pada Rumah Kaca,"Hairil Budiarto, S.T., M.T.","Faikul Umam, S.Kom., M.T.",Pembuatan garam di Madura umumnya dilakukan de...,Teknik Mekatronika,Making salt in Madura is generally done by hea...
101,ADINDA DEBTIANA DWIKA HILDA,KESTABILAN KECEPATAN MOBILE ROBOT PADA LINTASA...,"Faikul Umam, S.Kom., M.T.","Sri Wahyuni, S.Kom., M.T.",Mobile robot merupakan salah satu kategori rob...,Teknik Mekatronika,Mobile robot is one of the robot categories th...
102,Nafizatul Jamilah,Rancang Bangun Sistem Otomasi Robot Pengecatan...,"Hairil Budiarto, S.T., M.T","Sri Wahyuni, S.Kom., M.T",ABSTRAK\n\nMarka jalan raya adalah sebuah tand...,Teknik Mekatronika,ABSTRACT\n\nA highway mark is a sign that can ...


## kode untuk mengambil berita dari sebuah website berita

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import json

def scrap_semua_berita_liputan6(max_articles=200):
    """
    Mengambil berita dari indeks Liputan6.com hingga jumlah artikel maksimal tercapai.
    """
    base_url = "https://www.liputan6.com/indeks"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    data = {
        "id": [],
        "judul_berita": [],
        "kategori_berita": [],
        "isi_berita": []
    }

    id_counter = 1
    page_counter = 1

    # --- PERUBAHAN UTAMA: Loop terus berjalan selama target artikel belum tercapai ---
    while id_counter <= max_articles:
        url = f"{base_url}?page={page_counter}"
        print(f"Mengambil data dari halaman: {page_counter} (Total artikel terkumpul: {id_counter-1})")

        try:
            r = requests.get(url, headers=headers)
            soup = BeautifulSoup(r.content, "html.parser")
            list_artikel = soup.select("article.articles--rows--item")

            # Berhenti jika halaman sudah tidak mengandung artikel (halaman terakhir)
            if not list_artikel:
                print("Tidak ada artikel lagi ditemukan. Proses berhenti.")
                break

            for artikel in list_artikel:
                # Berhenti jika jumlah artikel sudah mencapai batas maksimal
                if id_counter > max_articles:
                    break

                try:
                    link_detail_tag = artikel.select_one("a.articles--rows--item__title-link")
                    if not link_detail_tag or not link_detail_tag.has_attr('href'): continue

                    link_detail = link_detail_tag['href']
                    if not link_detail.startswith("https://www.liputan6.com"): continue

                    response = requests.get(link_detail, headers=headers)
                    soup_detail = BeautifulSoup(response.content, "html.parser")

                    judul = "Judul tidak ditemukan"
                    kategori = "Kategori tidak ditemukan"
                    script_tag = soup_detail.find("script", string=re.compile("window.kmklabs.gtm"))

                    if script_tag:
                        script_content = script_tag.string
                        match_title = re.search(r'"articleTitle":"(.*?)"', script_content)
                        if match_title:
                            judul = match_title.group(1).encode('utf-8').decode('unicode-escape')

                        match_cat = re.search(r'window\.kmklabs\.category\s*=\s*({.*?});', script_content)
                        if match_cat:
                            cat_data_str = match_cat.group(1)
                            try:
                                kategori = json.loads(cat_data_str).get("name", "Kategori tidak ditemukan")
                            except json.JSONDecodeError: pass

                    isi_container = soup_detail.select_one("div.article-content-body__item-content")
                    isi_berita = ' '.join([p.get_text(strip=True) for p in isi_container.find_all("p")]) if isi_container else "Isi berita tidak ditemukan."

                    if judul != "Judul tidak ditemukan":
                        data["id"].append(id_counter)
                        data["judul_berita"].append(judul)
                        data["kategori_berita"].append(kategori)
                        data["isi_berita"].append(isi_berita)
                        id_counter += 1

                    time.sleep(0.5)
                except Exception:
                    continue

            page_counter += 1 # Pindah ke halaman berikutnya
            time.sleep(1)
        except Exception:
            break

    df = pd.DataFrame(data)
    df.to_csv("semua_berita_liputan6.csv", index=False)
    return df

In [32]:
scrap_semua_berita_liputan6()


Mengambil data dari halaman: 1 (Total artikel terkumpul: 0)
Mengambil data dari halaman: 2 (Total artikel terkumpul: 8)
Tidak ada artikel lagi ditemukan. Proses berhenti.


,id,judul_berita,kategori_berita,isi_berita
0,1,komisi-i-dpr-minta-polisi-taat-hukum-soal-renc...,Peristiwa,"Liputan6.com, Jakarta-Wakil Ketua Komisi I DPR..."
1,2,alasan-tersangka-kasus-penculikan-dan-pembunuh...,Peristiwa,"Liputan6.com, Jakarta-Salah satu tersangka kas..."
2,3,jk-soroti-disparitas-di-dki-orang-kaya-dan-mis...,Megapolitan,"Liputan6.com, Jakarta -Ketua UmumPalang Merah ..."
3,4,heboh-tanggul-beton-di-cilincing-pemprov-dki-i...,Megapolitan,"Liputan6.com, Jakarta -Pemerintah Provinsi (Pe..."
4,5,komisi-xi-minta-kemenkeu-dalam-kelola-apbn-tun...,Peristiwa,"Liputan6.com, Jakarta -Wakil Ketua Komisi XI D..."
5,6,mengenal-lebih-dekat-pppk-paruh-waktu-definisi...,Peristiwa,"Liputan6.com, Jakarta -Pemerintah Indonesia te..."
6,7,panduan-lengkap-sscasnbkngoid-persiapan-pendaf...,Peristiwa,"Liputan6.com, Jakarta -Portal sscasn.bkn.go.id..."
7,8,bsu-kemnaker-2025-panduan-lengkap-syarat-dan-c...,Peristiwa,"Liputan6.com, Jakarta -Pemerintah Indonesia me..."


## kode uuntuk mengambil semua link yang ada di sebuah website dan mencari lagi semua link yang ada di dalamnya smpai semua telah terjelajahi

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse
import time
from collections import deque

def find_all_links_pta(start_url, max_pages=50):
    """
    Menjelajahi situs PTA Trunojoyo dan mencatat SEMUA link unik yang ditemukan,
    tanpa membedakan internal atau eksternal.
    """
    try:
        start_domain = urlparse(start_url).netloc
    except ValueError:
        return pd.DataFrame(columns=["id", "page_sumber", "link_ditemukan"])

    urls_to_visit = deque([start_url])
    visited_urls = set()
    unique_links_found = set() # Menggunakan set untuk mencegah duplikasi link
    output_data = []
    link_id = 1

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    while urls_to_visit and len(visited_urls) < max_pages:
        current_url = urls_to_visit.popleft()
        if current_url in visited_urls:
            continue

        visited_urls.add(current_url)
        time.sleep(0.1)

        try:
            response = requests.get(current_url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'lxml')

            for link in soup.find_all('a', href=True):
                href = link['href']

                if not href or href.startswith(('#', 'mailto:', 'tel:', 'javascript:')):
                    continue

                full_url = urljoin(current_url, href)

                # Cek jika link ini belum pernah kita temukan sebelumnya
                if full_url not in unique_links_found:
                    unique_links_found.add(full_url)
                    output_data.append({
                        "id": link_id,
                        "page_sumber": current_url,
                        "link_ditemukan": full_url
                    })
                    link_id += 1

                # Jika link adalah internal, tambahkan ke antrean untuk dijelajahi
                link_domain = urlparse(full_url).netloc
                if link_domain == start_domain and full_url not in visited_urls:
                    if full_url not in urls_to_visit:
                         urls_to_visit.append(full_url)

        except requests.exceptions.RequestException:
            continue

    # MEKANISME PENYIMPANAN OUTPUT
    df = pd.DataFrame(output_data, columns=["id", "page_sumber", "link_ditemukan"])
    df.to_csv("semua_link_pta.csv", index=False)
    return df

In [29]:
# Tentukan website target
target_website = "https://pta.trunojoyo.ac.id/"

# Panggil fungsi (akan berjalan senyap)
find_all_links_pta(target_website)


,id,page_sumber,link_ditemukan
0,1,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/index.html
1,2,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/
2,3,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/c_search/
3,4,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/c_template/
4,5,https://pta.trunojoyo.ac.id/,https://library.trunojoyo.ac.id/detil.php?id=23
...,...,...,...
378,379,https://pta.trunojoyo.ac.id/c_search/byfac/7,https://pta.trunojoyo.ac.id/welcome/detail/110...
379,380,https://pta.trunojoyo.ac.id/c_search/byfac/7,https://pta.trunojoyo.ac.id/welcome/detail/110...
380,381,https://pta.trunojoyo.ac.id/c_search/byfac/7,https://pta.trunojoyo.ac.id/c_search/byfac/7/2
381,382,https://pta.trunojoyo.ac.id/c_search/byfac/7,https://pta.trunojoyo.ac.id/c_search/byfac/7/3
